# Generación de Datasets para el Analisis Exploratorio

In [39]:
# Importacion de paquetes necesarios
import requests
import json
import pandas as pd
import os


#### En archivo config.json tengo los datos de condifiguración para generar el ACCESS_TOKEN, necesario para hacer las querys

In [50]:
try:
    with open('config/config.json') as file:
        settings = json.load(file)
except Exception as e:
    print(e)
    
APP_ID= settings["APP_ID"]
SECRET_KEY= settings["SECRET_KEY"]
REDIRECT_URI= settings["REDIRECT_URI"]


#### Accediendo a este Link obtengo el SERVER_GENERATED_AUTHORIZATION_CODE que se agrega al archivo de configuración para luego hacer 

In [51]:
(f"http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id={APP_ID}&redirect_uri={REDIRECT_URI}")


'http://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=4736838571701537&redirect_uri=https://www.google.com/'

In [46]:
SERVER_GENERATED_AUTHORIZATION_CODE= "TG-5f20a9d52eb11d0007c88e08-46180769"
response = requests.post(url = f"https://api.mercadolibre.com/oauth/token?grant_type=authorization_code&client_id={APP_ID}&client_secret={SECRET_KEY}&code={SERVER_GENERATED_AUTHORIZATION_CODE}&redirect_uri={REDIRECT_URI}")
item = json.loads(response.content)
print(item)
ACCESS_TOKEN = item["access_token"]


{'access_token': 'APP_USR-4736838571701537-072822-b196993b4178c94c70994dce6ccdd7db-46180769', 'token_type': 'bearer', 'expires_in': 21600, 'scope': 'offline_access read write', 'user_id': 46180769, 'refresh_token': 'TG-5f20a9de04fd910006cee00a-46180769'}


#### Primero genero el json que contenga todos los datos agrupados de las publicaciones totales, publicaciones solo con descuentos y publicaciones sin descuentos, cruzando las Ids de las categorias con los nombres reales.

#### Esta proceso 2 veces: el 21 de Julio (Antes del HOTSALE en la carpeta data_analisis) y el 27 de Julio (Despues del HOTSALE en la carpeta data_analisis_HS) para comparar el estado de las publicaciones con descuentos en dos momentos muy disitntos y encontrar insights interesantes.

In [ ]:
folder = "data_analisis"
category_names = requests.get("https://api.mercadolibre.com/sites/MLA/categories")
category_full = json.loads(category_names.content)

for n,category_id in enumerate(category_full):
    category_discount = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?category={category_id['id']}&access_token={ACCESS_TOKEN}&discount=1-100")
    category_total = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?category={category_id['id']}&access_token={ACCESS_TOKEN}")
    category_nodiscount = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?category={category_id['id']}&access_token={ACCESS_TOKEN}&discount=0")
    category_discount = json.loads(category_discount.content)["paging"]["total"]
    category_total = json.loads(category_total.content)["paging"]["total"]
    category_nodiscount = json.loads(category_nodiscount.content)["paging"]["total"]
    category = category_full[n]
    category["total_discount"] = category_discount  
    category["total"] = category_total
    category["total_nodiscount"] = category_nodiscount
with open(os.path.join(folder , 'categorias.json'), 'w') as fp:
    json.dump(category_full, fp)

In [32]:
df_categ = pd.DataFrame.from_dict(category_full)
df_categ["ratio_discount"] = 100*df_categ["total_discount"]/df_categ["total"]

In [33]:
df_categ

,id,name,total_discount,total,total_nodiscount,ratio_discount
0,MLA5725,Accesorios para Vehículos,194678,8103580,7908904,2.402370
1,MLA1512,Agro,2610,54318,51709,4.805037
2,MLA1403,Alimentos y Bebidas,4272,187652,183382,2.276554
3,MLA1071,Animales y Mascotas,635,149797,149163,0.423907
4,MLA1367,Antigüedades y Colecciones,403,511419,511016,0.078800
5,MLA1368,"Arte, Librería y Mercería",3449,378195,374746,0.911963
6,MLA1743,"Autos, Motos y Otros",0,185225,185225,0.000000
7,MLA1384,Bebés,10685,299015,288322,3.573399
8,MLA1246,Belleza y Cuidado Personal,29400,427362,397965,6.879414
9,MLA1039,Cámaras y Accesorios,1525,167515,165990,0.910366


#### Se armó el dataset de publicaciones con el servicio https://api.mercadolibre.com/sites/MLA/search?category= para buscar todas las publicaciones por categoria y movernos con el offset cada 50 resultados. Debido a que hay un limite en el tamaño máximo de offset que uno puede usar, se dividio las querys en % de descuento tambien para que ninguna query supere los 10.000 resultados.

#### En este caso el dataset solo se obtuvo para el 21 de Julio y solo se analizara en esta fecha. Este momento previo al Hotsale es más representativo para el resto del año. Este proceso tarda varias horas, por lo que se recomienda empezar corriendo desde Exploration.ipnb ya con el csv más manejable

In [ ]:
fulllist = []
discount_jump = 1
for discount in range(1,100,discount_jump): 
    for categ_id in df_categ["id"]:
        quantity = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?category={categ_id}&offset=0&access_token={ACCESS_TOKEN}&discount={discount}-{discount+discount_jump}")
        quantity = json.loads(quantity.content)["paging"]["total"]
        categ_results =[]
        for offset in range(0, 10000 if quantity >10000  else quantity ,50  ):
            response = requests.get(f"https://api.mercadolibre.com/sites/MLA/search?category={categ_id}&offset={offset}&access_token={ACCESS_TOKEN}&discount={discount}-{discount+discount_jump}")
            item = json.loads(response.content)
            [x.update({"category_main_id":categ_id}) for x in item["results"]]
            categ_results = item["results"] + categ_results
        fulllist = fulllist+ categ_results
    print(f"discount={discount}-{discount+discount_jump}",  "Total :", len(fulllist) )
len(fulllist)
with open(folder + "discount.json", "w") as outfile: 
    json.dump(fulllist, outfile) 

### Armo una función que pode inicialmente caracteristicas poco interesantes.

In [ ]:
# with open(folder + "discount.json", "r") as infile: 
#     discount = json.load(infile) 

In [ ]:
def mowing_json(dict_input):
    dict_mowing  = dict_input.copy()
    dict_mowing.pop("attributes", None)
    dict_mowing.pop("permalink", None)
    dict_mowing["seller"].pop("eshop", None)
    dict_mowing["seller"].pop("permalink", None)
    dict_mowing["seller"].pop("id", None)
    dict_mowing["seller"].get("seller_reputation",{}).pop("metrics", None)
    dict_mowing["seller"].pop("tags", None)
    dict_mowing["seller"].pop("car_dealer_logo", None)
    dict_mowing.pop("site_id", None)
    dict_mowing.pop("thumbnail", None)
    dict_mowing.pop("seller_contact", None)
    dict_mowing.pop("official_store_id", None) 

    return dict_mowing

## Extiendo el tratamiento del json para reducir su tamaño:
 - Conversion jason a DataFrame
 - Eliminación de registros repetidos
 - Creación de la variable Discount
 - Guardado a CSV

In [ ]:
df_json = [mowing_json(x) for x in discount] # Aplica funcon de podado en Json
df = pd.json_normalize(df_json,sep="_") # Transformo Json a DataFrame
df = df[df.drop(["tags","shipping_tags"],axis=1).duplicated()].reset_index(drop=True) # Elimino registros duplicados
df = df.infer_objects()  # Inferimos el mejor tipo de dato para la columna
df = df.replace('', np.nan) # Reemplazo espacios vacios por NaN
df["discount"] = 100- (100 * df["price"] / df["original_price"])
df = df.drop(df[df["discount"].isnull()].index)
df = df[df["category_main_id"].isin(list((categorias[categorias["total_discount"]>2])["id"]))] # Filtro las categorias con menos de 2 publicaciones con Descuento
df.to_csv("data_analisis/discount.csv") # Guardo dataset en CSV